In [2]:
import requests
from io import BytesIO
from PIL import Image
import os

from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import torch
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset

from DataHandlers import ImageDataset, InMemDataLoader

/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.9/site-packages/torchvision/image.so, 0x0006): symbol not found in flat namespace '__ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [24]:
class Wrapper():
    def __init__(
                self, 
                model, 
                paths = {'train': 'data/train.csv', 'valid': 'data/valid.csv', 'test': 'data/test.csv'},
                sample_sizes = {'train': 100, 'valid': 100, 'test': 100},
                transform = None,
                batch_size = 20,
                inMemLoaders = False,
            ):
        
        self.model = model
        self.paths = paths
        self.sample_sizes = sample_sizes
        self.batch_size = batch_size
        self.n = {}
        
        def _create_metadata(dataset_type):
            data = pd.read_csv(paths[dataset_type])[:sample_sizes[dataset_type]]
            self.n[dataset_type] = len(data)
            return data

        def _create_dataloader(dataset, shuffle):
            if inMemLoaders:
                return InMemDataLoader(
                    dataset,
                    
                    batch_size=batch_size,
                    shuffle=shuffle,
                )
            else:
                return DataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle,
                )
        
        if transform is None:
            transform = T.Compose([
                T.Resize(200),
                T.CenterCrop(200),
                T.ToTensor(),
                T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

        dataset_types = ['train', 'valid', 'test']

        self.datasets = {
            dataset_type: ImageDataset(_create_metadata(dataset_type), transform=transform)
            for dataset_type in dataset_types
        }

        self.loaders = {
            dataset_type: _create_dataloader(dataset, shuffle=(dataset_type=='train'))
            for dataset_type, dataset in self.datasets.items()
        }
        
        
    def train(self):
        self.model.train(self.loaders['train'])
        
    def accuracy(self):
        # train_preds = self.model.predict(self.loaders['valid'])
        def _count(loader):
            preds, truth = [], []
            for images1, images2, equal in loader:
                preds.append(self.model.predict(images1, images2))
                truth.append(equal)

            preds = torch.cat(preds)
            truth = torch.cat(truth)
            return (preds == truth).float().mean()
    
        print(f'Train accuracy : {_count(self.loaders["train"])}\n') 
        print(f'Validation accuracy : {_count(self.loaders["valid"])}\n') 
    
        
    def save_test_preds(self, path='res.csv'):
        print(f'Started saving test predictions to {path}')
        ids = []
        preds = []
            
        for images1, images2, id_ in tqdm(self.loaders['test']):
            preds.extend(self.model.predict(images1, images2))
            ids.extend(id_)
            
        all_ids = pd.DataFrame({
            'ID': range(2, self.n['test']),
        })
        res = pd.DataFrame({
            'ID': [obj.item() for obj in ids],
            'is_same': [obj.item() for obj in preds]
        }).drop_duplicates()

        res = all_ids.merge(res, on='ID', how='left').fillna(0)
        res.to_csv(path, index=False)
        print(f'Saved test predictions to {path}\n')

In [25]:
class ThresholdClassifier():
    def __init__(self):
        pass
    
    def train(self, loader):
        pass
    
    def predict(self, images1, images2):
        delta = ((images1 - images2) ** 2).mean(dim=(1,2,3))
        preds = (delta < 0.02) * 1
        return preds

In [26]:
model = ThresholdClassifier()
wrapper = Wrapper(model)
wrapper.train()
wrapper.accuracy()
wrapper.save_test_preds()

Train accuracy : 0.949999988079071

Validation accuracy : 0.949999988079071

Started saving test predictions to res.csv


  0%|          | 0/5 [00:00<?, ?it/s]

Saved test predictions to res.csv



In [12]:
train = pd.read_csv('data/train.csv')
train = train.drop_duplicates()
train.to_csv('data/train.csv', index=False)